In [7]:
class Stalk:
    def __init__(self,stalk):
        self.stalk = stalk
        if len(self.stalk) == 0: self.stalk = '$'

    def __repr__(self): return self.stalk

    def __eq__(self,other): return self[0] == str(other)[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.stalk[index]

    def __len__(self):
        if self.stalk == '$' or self.stalk == '^': return 0
        return len(self.stalk)
    
    def common_substring(self,other):
        i = 0
        substr = ''
        if type(other) == str: other = Stalk(other)
        while self[i] == other[i] and i < min(len(self),len(other)):
            substr += self[i]
            i += 1
        return Stalk(substr), Stalk(self[i:]), Stalk(other[i:])

In [8]:
class Leaf:
    def __init__(self,left,right=''):
        if len(left) == 0:
            self.left = left
            self.right = 1
            # self.reads = [read]
        else:
            # self.reads = [read]
            self.left = left
            self.right = Leaf(right)
    
    def __repr__(self): return '(' + repr(self.left) + ',' + repr(self.right) + ')'

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self.left)

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)+len(self.right)

In [9]:
class Branch:
    def __init__(self):
        self.b = {}
        self.stalks = {}
        # self.reads = []

    def __repr__(self): 
        s = ''
        for a in self.b.values(): 
            s += repr(a) 
        return s

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index):
        if type(index) == str: return self.b[Stalk(index)]
        return self.b[index]
    
    def __setitem__(self,index,value):
        if type(index) == str: self.b[Stalk(index)] = value
        else: self.b[index] = value

    def __contains__(self,other): 
        if type(other) == str: return self.b[Stalk(other)]
        return other in self.b

    def __len__(self): return len(self.b)

    def pop(self,index): return self.pop(index)

    # def is_shallow(self):
    #     for x in self.b.values():
    #         if type(x.right) == Leaf or x.left == '$': continue
    #         else: 
    #             for y in x.right.b.values():
    #                 if type(y.right) == Leaf or y.left == '$': continue
    #                 else: return False
        # return True

    def add(self,stalk,reads):
        if stalk in self:
            if type(self[stalk]) == Leaf:
                branch = Branch()
                l1 = self.pop(stalk)
                stalk_ = list(self.stalks.pop(stalk))
                stalk_[0],l1.left,l2 = stalk_[0].common_string(stalk)
                branch.add(l1,stalk_[1])
                l2 = Leaf(l2)
                stalk_[1] += reads
                branch.add(l2,reads)
                stalk_ = tuple(stalk_)
                self[stalk_[0]] = branch
                self.stalks[stalk_[0]] = stalk_
            else:
                stalk_ = list(self.stalks.pop(stalk))
                branch = self.pop(stalk)
                stalk_[0],bstalk,stalk = stalk_[0].common_string(stalk)
                if len(bstalk):
                    br = Branch()
                    br[bstalk] = branch 
                    br.stalks[bstalk] = (bstalk,stalk_[1])
                    br.add(stalk,reads)
                    self[stalk_[0]] = br
                else: branch.add(stalk,reads)
                stalk_[1] += reads
                stalk_ = tuple(stalk_)
                self.stalks[stalk_[0]] = stalk_
        else:
            if type(stalk) == str: stalk = Stalk(stalk)
            self.stalks[stalk] = (stalk,reads)
            self[stalk] = Leaf(stalk)
            # self.reads += [read]

In [ ]:
# class Branch:
#     def __init__(self):
#         self.b = {}
#         self.reads = []

#     def __repr__(self): 
#         s = ''
#         for a in self.b.values(): 
#             s += repr(a) 
#         return s

#     def __str__(self):
#         s = '' 
#         for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
#         return s+str(list(self.b.values())[-1])
    
#     def __getitem__(self,index): return self.b[index]
    
#     def __contains__(self,other): return other in self.b

#     def __len__(self): return len(self.b)

#     # def is_shallow(self):
#     #     for x in self.b.values():
#     #         if type(x.right) == Leaf or x.left == '$': continue
#     #         else: 
#     #             for y in x.right.b.values():
#     #                 if type(y.right) == Leaf or y.left == '$': continue
#     #                 else: return False
#         # return True

#     def add(self,leaf,read):
#         if leaf in self.b:
#             if self.b[leaf].left == '$':
#                 self.b[leaf].right+=1
#                 return
#             i = 0
#             left = ''
#             while i < min(len(leaf.left),len(self.b[leaf].left)) and leaf.left[i] == self.b[leaf][i]: 
#                 left += leaf.left[i]
#                 i += 1
#             if type(self.b[leaf].right) == Branch:
#                 nleaf = Leaf(leaf.origin,leaf.left[i:])
#                 if len(self.b[leaf].left[i:]):
#                     branch = Branch()
#                     l = Leaf(self.b[leaf].origin,self.b[leaf].left[i:])
#                     l.right = self.b[leaf].right
#                     branch.b[l] = l
#                     branch.reads += [l.origin]
#                     self.b[leaf].right = branch
#                 self.b[leaf].right.add(nleaf,leaf.origin)
#             else: 
#                 branch = Branch()
#                 branch.add(Leaf(self.b[leaf].origin,self.b[leaf].left[i:]),self.b[leaf].origin)
#                 branch.add(Leaf(leaf.origin,leaf.left[i:]),leaf.origin)
#                 self.b[leaf].right = branch
#             self.b[leaf].left = left
#             self.reads += [read]
#         else: 
#             self.b[leaf] = leaf
#             self.reads += [read]

In [3]:
class Sequence:
    def __init__(self,seq=None,initus=None,terminus=None):
        if seq is None: self.seq = ''
        else: self.seq = seq
        if initus is None: self.initus = ''
        else: self.initus = initus
        if terminus is None: self.terminus = ''
        else: self.terminus = terminus

In [5]:
class Sequitur:
    def __init__(self,reads):
        self.branch = Branch()
        self.reads = reads
        for read in reads: 
            for i in range(len(read)): self.branch.add(read[i:],read)
        self.sequence = ''

    def _traverse(self,context):
        branch = self.branch.b[context[0]]
        context = context[len(branch.left):]
        while len(context) > 0:
            # if q[0] not in branch.right:
            #     print sequence.seq
            #     print(branch)
            branch = branch.right[context[0]]
            context = context[len(branch.left):]
        return branch

    def sequitur(self,sequence):
        i = 1
        context = sequence.seq[-i]
        options = []
        while i <= len(sequence.seq):
            branch = self._traverse(context)
            i += 1
            context = sequence.seq[-i:]

    def assemble(self,start=None):#,seed=None,correct_sequence=None):
        pass